In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI Platform (Unified) SDK: AutoML tabular classification model for online prediction

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/blob/master/notebooks/deepdive/automl/tabular/ucaip_automl_tabular_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/notebooks/deepdive/automl/tabular/ucaip_automl_tabular_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>
<br/><br/><br/>

# Overview


This tutorial demonstrates how to use the AI Platform (Unified) Python SDK to create tabular models using Google Cloud's [AutoML Tabular](https://cloud.google.com/automl-tables).


### Dataset

The dataset used for this tutorial is the [Iris dataset](https://www.tensorflow.org/datasets/catalog/iris) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview). This dataset does not require any feature engineering. The version of the dataset you will use in this tutorial is stored in a public Cloud Storage bucket.


### Objective

In this notebook, you will learn how to create a tabular classification model with AutoML Tabular from a Python script using the AI Platform (Unified) SDK. You can alternatively create models with AutoML Tabular from the command line using `gcloud` or online using Google Cloud Console.

The steps performed include: 

- Create a AI Platform (Unified) managed Dataset.
- Train the model for up to one hour.
- View the model evaluation.
- Deploy the model to a serving endpoint.
- Make a prediction(s).
- Undeploy the model.

### Costs 

This tutorial uses billable components of Google Cloud Platform (GCP):

* Cloud AI Platform
* Cloud Storage

Learn about [Cloud AI Platform
pricing](https://cloud.google.com/ml-engine/docs/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the latest (alpha) version of AI Platform (Unified) SDK from a tar file we have in a GCP storage bucket.

**{Google Staff: When public, replace this with pip install from PyPi distribution}**

In [ ]:
! pip3 install https://storage.googleapis.com/google-cloud-aiplatform/libraries/python/0.1.1/google-cloud-aiplatform-0.1.1.tar.gz

Install Google cloud-storage as well.

In [ ]:
! pip3 install google-cloud-storage

### Restart the Kernel

Once you've installed the AI Platform (Unified) SDK and Google cloud-storage, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

**Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select Runtime --> Change runtime type -> GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in AI Platform Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Project ID

**If you don't know your project ID**, try to get your project ID using `gcloud` command by executing the second cell below.

In [1]:
PROJECT_ID = "[your-project-id]" #@param {type:"string"}

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: andy-1234-221921


In [3]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Cloud
AI Platform services are
available](https://cloud.google.com/ml-engine/docs/tensorflow/regions). You may
not use a Multi-Regional Storage bucket for training with AI Platform.

In [4]:
REGION = 'us-central1' #@param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, we create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.

In [5]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

*Note, if you are on AI Platform notebook and run the cell, the cell knows to skip executing the authentication steps.*

In [6]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AI Platform, then don't execute this code
if not os.path.exists('/opt/deeplearning/metadata/env_version'):
    if 'google.colab' in sys.modules:
        from google.colab import auth as google_auth
        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

### Create a Cloud Storage bucket

**The following steps are required if your data is in your own local Cloud Storage bucket, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets. 

In [7]:
local = False
if local:
    BUCKET_NAME = "[your-bucket-name]" #@param {type:"string"}

In [8]:
if local:
    if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
        BUCKET_NAME = PROJECT_ID + "ucaip-automl-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [9]:
if local:
    ! gsutil mb -l $REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [10]:
if local:
    ! gsutil ls -al gs://$BUCKET_NAME

### Set up variables

Let's set up some variables used to create an AutoML model.

### Import libraries and define constants

#### Import AI Platform (Unified) SDK

Import the AI Platform (Unified) SDK into our python environment.

In [11]:
import os
import sys
import time

from google.cloud import aiplatform_v1alpha1 as aip

#### AI Platform (Unified) constants

Let's now setup some constants for AutoML:

- `API_ENDPOINT`: The AI Platform (Unified) API service endpoint for dataset, model, job, pipeline and endpoint services.
- `API_PREDICT_ENDPOINT`: The AI Platform (Unified) API service endpoint for prediction.
- `PARENT`: The AI Platform (Unified) location root path for dataset, model and endpoint resources.

In [12]:
# API Endpoint
API_ENDPOINT = "us-central1-aiplatform.googleapis.com"
API_PREDICT_ENDPOINT = "us-central1-prediction-aiplatform.googleapis.com"

# AI Platform (Unified) location root path for your dataset and model resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML constants

Let's now setup some constants for AutoML:

- Dataset Schemas - tells the managed dataset service which type of dataset it is.
- Data Labeling (Annotations) Schemas - tells the managed dataset service how the data is labeled (annotated).
- Dataset Training Schemas - tells the managed pipelines service the task (e.g., classification) to train the model for.

In [13]:
# Tabular Dataset type
TABLE_SCHEMA = 'google-cloud-aiplatform/schema/dataset/metadata/tables_1.0.0.yaml'
# Tabular Labeling type
IMPORT_SCHEMA_TABLE_CLASSIFICATION = 'gs://google-cloud-aiplatform/schema/dataset/ioformat/table_io_format_1.0.0.yaml'
# Tabular Training task
TRAINING_TABLE_CLASSIFICATION_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_tables_1.0.0.yaml"

# Tutorial

Now you are ready to start creating your own AutoML Tabular model for tabular classification.

## Clients

The AI Platform (Unified) SDK works as a client/server model. On your side, the Python script, you will create a client that sends requests and receives responses from the server -- AI Platform.

You will use several clients in this tutorial, so you will set them all up upfront.

- Dataset Service for managed datasets.
- Model Service for managed models.
- Pipeline Service for training.
- Endpoint Service for deployment.
- Prediction Service for serving. *Note*, prediction has a different service endpoint.

In [14]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}
predict_client_options = {"api_endpoint": API_PREDICT_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(
        client_options=client_options
    )
    return client


def create_model_client():
    client = aip.ModelServiceClient(
        client_options=client_options
    )
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(
        client_options=client_options
    )
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(
        client_options=client_options
    )
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(
        client_options=predict_client_options
    )
    return client


clients = {}
clients['dataset'] = create_dataset_client()
clients['model'] = create_model_client()
clients['pipeline'] = create_pipeline_client()
clients['endpoint'] = create_endpoint_client()
clients['prediction'] = create_prediction_client()

for client in clients.items():
    print(client)

('dataset', <google.cloud.aiplatform_v1alpha1.services.dataset_service.client.DatasetServiceClient object at 0x7f8d796c1d10>)
('model', <google.cloud.aiplatform_v1alpha1.services.model_service.client.ModelServiceClient object at 0x7f8d796c1ad0>)
('pipeline', <google.cloud.aiplatform_v1alpha1.services.pipeline_service.client.PipelineServiceClient object at 0x7f8d796c1d50>)
('endpoint', <google.cloud.aiplatform_v1alpha1.services.endpoint_service.client.EndpointServiceClient object at 0x7f8d796c1e10>)
('prediction', <google.cloud.aiplatform_v1alpha1.services.prediction_service.client.PredictionServiceClient object at 0x7f8d796c1dd0>)


## Dataset 

Now that your clients are ready, your first step is to create a managed dataset instance. This step differs from Vision, Video and Language. For those products, after the managed dataset is created, one then separately imports the data, using the `import_data` method.

For tabular, importing of the data is deferred until the training pipeline starts training the model. What do we do different? Well, first you won't be calling the `import_data` method. Instead, when you create the dataset instance you specify the Cloud Storage location of the CSV file, which contains your tabular data as part of the managed dataset's metadata.

`metadata = {"input_config": {"gcs_source": {"uri": [gcs_uri]}}}`

Note that the `gcs_source` field is a list, whereby you can input multiple CSV files when your data is split across files. 

### Location of Cloud Storage training data.

Let's now set the variable `IMPORT_FILE` to the location of the CSV file in Cloud Storage.

Set the local variable `IMPORT_FORMAT` to indicate whether your dataset is a CSV index file.

Additionally, you can set the variable `SPLIT_TYPE` to choose how AutoML will handle splitting the dataset into training, test and validation sets:

- DEFAULT - AutoML chooses the split.
- ML_USE - Examples are tagged which set they below to (TRAINING, TEST, VALIDATION).
- FRACTION - Percentage split ratios specified in `input_config` when training.

In [15]:
# Tabular Classification
# No Split
IRIS_CSV = 'gs://cloud-samples-data/tables/iris_1000.csv'
# ML_USE split
IRIS_SPLIT_CSV = 'gs://cloud-samples-data/tables/iris_1000-split.csv'

IMPORT_FORMAT = 'CSV'  # [CSV]
SPLIT_TYPE = 'DEFAULT'  # [ML_USE, FRACTION, DEFAULT]

if IMPORT_FORMAT == 'CSV':
    if SPLIT_TYPE == 'ML_USE':
        IMPORT_FILE = IRIS_SPLIT_CSV
    else:
        IMPORT_FILE = IRIS_CSV

### Create a managed dataset instance

Use this helper function `create_dataset` to create the instance of your managed dataset. This function does:

1. Uses the dataset client service.
2. Creates a AI Platform (Unified) dataset object (`aip.Dataset`), with the parameters:
- `display_name`: The human-readable name you choose to give it, and
- `metadata_schema_uri`: The dataset type. For this tutorial this will be the schema for tabular dataset type.
- `metadata`: The Cloud Storage location of the tabular data.
3. Calls the client dataset service method `create_dataset`, with the parameters:
- `parent`: AI Platform (Unified) location root path for your datase, model and endpoint resources.
- `dataset`: the AI Platform (Unified) dataset object instance you created.
4. The method returns an `operation` object.

An `operation` object is how AI Platform (Unified) handles asynchronous calls for long running operations. While this step usually goes fast, when you first use it in your project, there is a longer delay due to provisioning.

You can use the `operation` object to get status on the operation (e.g., create managed dataset) or to cancel the operation, by invoking an operation method:

| Method      | Description |
| ----------- | ----------- |
| result()    | Waits for the operation to complete and returns a result object in JSON format.      |
| running()   | Returns True/False on whether the operation is still running.        |
| done()      | Returns True/False on whether the operation is completed. |
| canceled()  | Returns True/False on whether the operation was canceled. |
| cancel()    | Cancels the operation (this may take up to 30 seconds). |


In [16]:
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value, Struct

TIMEOUT = 60
DATA_SCHEMA = TABLE_SCHEMA


def create_dataset(name, schema, gcs_uri=None, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        metadata = {"input_config": {"gcs_source": {"uri": [gcs_uri]}}}
        dataset = aip.Dataset(display_name=name, metadata_schema_uri="gs://" + schema, labels=labels,
                              metadata=json_format.ParseDict(metadata, Value()))

        operation = clients['dataset'].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        response = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", response.name)
        print(" display_name:", response.display_name)
        print(" metadata_schema_uri:", response.metadata_schema_uri)
        print(" metadata:", dict(response.metadata))
        print(" create_time:", response.create_time)
        print(" update_time:", response.update_time)
        print(" etag:", response.etag)
        print(" labels:", dict(response.labels))
        return {'name': response.name, 'schema': schema}
    except Exception as e:
        print("exception:", e)
        return (None, None)


dataset = create_dataset("automl-" + TIMESTAMP, DATA_SCHEMA, gcs_uri=IMPORT_FILE)

Long running operation: projects/759209241365/locations/us-central1/datasets/2773021101809205248/operations/86179721385082880
time: 2.395200490951538
response
 name: projects/759209241365/locations/us-central1/datasets/2773021101809205248
 display_name: automl-20201006171818
 metadata_schema_uri: gs://google-cloud-aiplatform/schema/dataset/metadata/tables_1.0.0.yaml
 metadata: {'inputConfig': <proto.marshal.collections.maps.MapComposite object at 0x7f8d7966cf90>}
 create_time: None
 update_time: None
 etag: 
 labels: {'aiplatform.googleapis.com/dataset_metadata_schema': 'TABLE'}


### Data preparation

The AI Platform (Unified) managed dataset for tabular has a couple of requirements for your tabular data.

- Must be in a CSV file.

#### CSV

For tabular classification, the CSV file has a few requirements:

- The first row must be the heading -- note how this is different from Vision, Video and Language where the requirement is no heading.
- All but one column are features.
- One column is the label, which you will specify when you subsequently create the training pipeline.

### Dataset splitting

#### CSV

Each row entry in a CSV file may be preceded by a first column that indicates whether the data is part of the training (TRAINING), test (TEST) or validation (VALIDATION) data.  Alternatively, AI Platform (Unified) supports the CAIP (pre-AI Platform (Unified)) version of the tags: TRAIN, TEST and VALIDATE. For example:

    TRAINING, "this is the data item", "this is the label"
    TEST, "this is the data item", "this is the label"
    VALIDATION, "this is the data item", "this is the label"
    
Otherwise, AutoML will automatically split the dataset for you.

#### Quick Peek at your data

You will use a version of the Iris dataset that is stored in a public Cloud Storage bucket, using a CSV file. 

Let's start by doing a quick peek at the data. You count the number of examples by counting the number of rows in the CSV file  (`wc -l`) and then peek at the first few rows.

You also need for training to know the heading name of the label column, which is save as `label_column`.

In [17]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
if IMPORT_FORMAT == 'CSV':
    label_column = str(heading).split(',')[-1].split("'")[0]
print("Label Column Name", label_column)

Number of Examples 1201
First 10 rows
TRAINING,sepal_length,sepal_width,petal_length,petal_width,species
TRAINING,4.5,2.3,1.3,0.3,setosa
TRAINING,4.5,2.3,1.3,0.3,setosa
TRAINING,4.5,2.3,1.3,0.3,setosa
TRAINING,4.5,2.3,1.3,0.3,setosa
TRAINING,4.5,2.3,1.3,0.3,setosa
TRAINING,4.5,2.3,1.3,0.3,setosa
TRAINING,4.5,2.3,1.3,0.3,setosa
TRAINING,4.5,2.3,1.3,0.3,setosa
TRAINING,5,2,3.5,1,versicolor
Label Column Name species


### Get dataset information

Now that the data is imported into your AI Platform (Unified) managed dataset, lets get some information about the current state of dataset. Use this helper function `get_dataset`, with the parameter:

- `name`: The AI Platform (Unified) fully qualified dataset identifier, which is in the form:

    projects/[project_id]/locations/[region]/datasets/[dataset id]

The helper function uses the dataset service client's method `get_dataset`, which takes as a parameter:

- `name`: The AI Platform (Unified) fully qualified dataset identifier.
    
If your recall, we got the fully qualified dataset identifier in the `name` field of the response object when we created the AI Platform (Unified) managed dataset instance.

The method returns an AI Platform (Unified) managed dataset object.

In [18]:
def get_dataset(name):
    response = clients['dataset'].get_dataset(name=name)
    print("TYPE", type(response))

    print("name:", response.name)
    print("display name:", response.display_name)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    print("labels:", response.labels)
    print("metadata_schema_uri:", response.metadata_schema_uri)
    print("metadata:", dict(response.metadata))


get_dataset(dataset['name'])

TYPE <class 'google.cloud.aiplatform_v1alpha1.types.dataset.Dataset'>
name: projects/759209241365/locations/us-central1/datasets/2773021101809205248
display name: automl-20201006171818
create_time: 2020-10-06 17:18:41.896134+00:00
update_time: 2020-10-06 17:18:42.529135+00:00
labels: {'aiplatform.googleapis.com/dataset_metadata_schema': 'TABLE'}
metadata_schema_uri: gs://google-cloud-aiplatform/schema/dataset/metadata/tables_1.0.0.yaml
metadata: {'inputConfig': <proto.marshal.collections.maps.MapComposite object at 0x7f8d796d2b50>}


### List all data items

Unlike image, vision and text, there is no dataitem concept in tabular.

## Train the model

Let's now train an AutoML tabular classification model using your AI Platform (Unified) managed dataset. To train the model, do the following steps:

1. Create a AI Platform (Unified) managed training pipeline for the dataset.
2. Execute the pipeline to start the training.

### Create a training pipeline

You may ask, what do we use a pipeline for? We typically use pipelines when the job (such as training) has multiple steps, generally in sequential order: do step A, do step B, etc. By putting the steps into a pipeline, we gain the benefits of:

1. Reusable for subsequent training jobs.
2. Can be containerized and ran as a batch job.
3. Can be distributed.
4. All the steps are associated with the same pipeline job for tracking progress.

Use this helper function `create_pipeline`, which takes the parameters:

- `pipeline_name`: A human readable name for the pipeline job.
- `model_name`: A human readable name for name the model.
- `dataset`: The AI Platform (Unified) fully qualified dataset identifier.
- `schema`: The dataset labeling (annotation) schema. For this tutorial, it will be the schema for training a tabular classification model.
- `task`: A dictionary describing the requirements for the training job.

The helper function uses the AI Platform (Unified) pipeline client service, calling the method `create_pipeline`, which takes the parameters:

- `parent`: The AI Platform (Unified) location root path for your dataset, model and endpoint resources.
- `training_pipeline`: The full specification for the pipeline training job.

Let's look now deeper into the *minimal* requirements for constructing a `training_pipeline` specification:

- `display_name`: A human readable name for the pipeline job.
- `training_task_definition`: The dataset labeling (annotation) schema.
- `training_task_inputs`: A dictionary describing the requirements for the training job.
- `input_data_config`: The dataset specification.
 - `dataset_id`: The AI Platform (Unified) dataset identifier only (non-fully qualified) -- this is the last part of the fully-qualified identifier.
 - `fraction_split`: If specified, the percentages of the dataset to use for training, test and validation. Otherwise, the percentages are automatically selected by AutoML.
- `model_to_upload`: A human readable name for name the model. 

In [19]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split('/')[-1]
    if SPLIT_TYPE == 'FRACTION':
        input_config = {'dataset_id': dataset_id,
                        'fraction_split': {
                            'training_fraction': 0.8,
                            'validation_fraction': 0.1,
                            'test_fraction': 0.1,
                        }}
    else:
        input_config = {'dataset_id': dataset_id}

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients['pipeline'].create_training_pipeline(parent=PARENT, training_pipeline=training_pipeline)
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

Next, you will construct the task requirements. Unlike other parameters which take a python (JSON-like) dictionary, the `task` field takes a Google protobuf Struct, which is very similar to a python dictionary. The minimal fields you need to specify are:

- `prediction_type`: Whether we are doing "classification" or "regression".
- `target_column`: The CSV heading column name for the column we want to predict (i.e., the label).
- `train_budget_milli_node_hours`: The maximum time to budget (billed) for training the model, where 1000 = 1 hour.
- `disable_early_stopping`: Whether True/False to let AutoML use its judgement to stop training early or train for the entire budget.
- `transformations`: Specifies the feature engineering for each feature column.

For `transformations`, the list must have an entry for each column. The outer key field indicates the type of feature engineering for the corresponding column. In this tutorial, you set it to `"auto"` to tell AutoML to automatically determine it.

Finally, you create the pipeline by calling the helper function `create_pipeline`, which returns an instance of a training pipeline object.

In [20]:
SCHEMA = TRAINING_TABLE_CLASSIFICATION_SCHEMA
PIPE_NAME = "iris_pipe-" + TIMESTAMP
MODEL_NAME = "iris_model-" + TIMESTAMP

TRANSFORMATIONS = [
    {"auto": {"column_name": "sepal_width"}},
    {"auto": {"column_name": "sepal_length"}},
    {"auto": {"column_name": "petal_length"}},
    {"auto": {"column_name": "petal_width"}}
]

task = Value(struct_value=Struct(
            fields={
                'target_column': Value(string_value=label_column),
                'prediction_type': Value(string_value="classification"),
                'train_budget_milli_node_hours': Value(number_value=1000),
                'disable_early_stopping': Value(bool_value=False),
                'transformations': json_format.ParseDict(TRANSFORMATIONS, Value())
            }
        ))
pipeline = create_pipeline(PIPE_NAME, MODEL_NAME, dataset['name'], SCHEMA, task)

name: "projects/759209241365/locations/us-central1/trainingPipelines/4219345085264822272"
display_name: "iris_pipe-20201006171818"
input_data_config {
  dataset_id: "2773021101809205248"
}
training_task_definition: "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_tables_1.0.0.yaml"
training_task_inputs {
  struct_value {
    fields {
      key: "predictionType"
      value {
        string_value: "classification"
      }
    }
    fields {
      key: "targetColumn"
      value {
        string_value: "species"
      }
    }
    fields {
      key: "trainBudgetMilliNodeHours"
      value {
        string_value: "1000"
      }
    }
    fields {
      key: "transformations"
      value {
        list_value {
          values {
            struct_value {
              fields {
                key: "auto"
                value {
                  struct_value {
                    fields {
                      key: "columnName"
                      value {
             

### List all training pipelines

Your training pipeline is now executing on Google Cloud AI Platform. Let's start by getting a list of all your pipelines and corresponding execution state. You likely only have one, but if you been experimenting with this tutorial or otherwise have used AI Platform (Unified) pipelines previously, you will see those as well.

Use this helper function `list_training_pipeline`. This function uses the pipeline client service and calls the method `list_training_pipelines`, with the parameter:

- `parent`: The AI Platform (Unified) location root path for your dataset, model and endpoint resources.

The method returns a `response object` as a list, where every element in the list is a pipeline object instance. The field we are most interest in is `response.state`, which should be at this early point: `PIPELINE_STATE_RUNNING` -- which means the model is being trained, but not completed. 

You could also see `PIPELINE_STATE_PENDING`, which indicates either the service has not yet finished provisioning the resources for the training job, or that the training job is momentarily been paused.

In [21]:
def list_training_pipeline():

    response = clients['pipeline'].list_training_pipelines(parent=PARENT)
    for pipeline in response:
        print("pipeline")
        print(" name:", pipeline.name)
        print(" display_name:", pipeline.display_name)
        print(" training_task_definition:", pipeline.training_task_definition)
        print(" training_task_inputs:", dict(pipeline.training_task_inputs))
        print(" state:", pipeline.state)
        print(" create_time:", pipeline.create_time)
        print(" start_time:", pipeline.start_time)
        print(" end_time:", pipeline.end_time)
        print(" update_time:", pipeline.update_time)
        print(" labels:", dict(pipeline.labels))


list_training_pipeline()

pipeline
 name: projects/759209241365/locations/us-central1/trainingPipelines/4219345085264822272
 display_name: iris_pipe-20201006171818
 training_task_definition: gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_tables_1.0.0.yaml
 training_task_inputs: {'targetColumn': 'species', 'transformations': [struct_value {
  fields {
    key: "auto"
    value {
      struct_value {
        fields {
          key: "columnName"
          value {
            string_value: "sepal_width"
          }
        }
      }
    }
  }
}
, struct_value {
  fields {
    key: "auto"
    value {
      struct_value {
        fields {
          key: "columnName"
          value {
            string_value: "sepal_length"
          }
        }
      }
    }
  }
}
, struct_value {
  fields {
    key: "auto"
    value {
      struct_value {
        fields {
          key: "columnName"
          value {
            string_value: "petal_length"
          }
        }
      }
    }
  }
}
, struct_value

### Get information on a training pipeline

Let's now get pipeline information for just this training pipeline instance. You will use the pipeline client service and invoke the `get_training_pipeline` method, with the parameter:

- `name`: The AI Platform (Unified) fully qualified pipelinne identifier.

When the model is done training, the pipeline state will be `PIPELINE_STATE_SUCCEEDED`.

In [30]:
def get_training_pipeline(name):
    response = clients['pipeline'].get_training_pipeline(name=name)

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


pipeline_response = get_training_pipeline(pipeline.name)

pipeline
 name: projects/759209241365/locations/us-central1/trainingPipelines/4219345085264822272
 display_name: iris_pipe-20201006171818
 state: PipelineState.PIPELINE_STATE_SUCCEEDED
 training_task_definition: gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_tables_1.0.0.yaml
 training_task_inputs: {'transformations': [struct_value {
  fields {
    key: "auto"
    value {
      struct_value {
        fields {
          key: "columnName"
          value {
            string_value: "sepal_width"
          }
        }
      }
    }
  }
}
, struct_value {
  fields {
    key: "auto"
    value {
      struct_value {
        fields {
          key: "columnName"
          value {
            string_value: "sepal_length"
          }
        }
      }
    }
  }
}
, struct_value {
  fields {
    key: "auto"
    value {
      struct_value {
        fields {
          key: "columnName"
          value {
            string_value: "petal_length"
          }
        }
      }
    }


# Deployment

## Pre-Cooked

Training the above model may take upwards of ~30 minutes time. For expendiency, we have a pre-cooked (already trained) version of this model you can use for the next steps, while you wait for your model to finish training. 

Once your model is done training, you can repeat these steps for your trained model. You can calcuate the actual time it took to train the model by subtracting `end_time` from `start_time`. For your model, we will need to know the fully qualified AI Platform (Unified) managed model identifier, which the pipeline service assigned to it. We can get this from the returned pipeline instance as the field `model_to_deploy.name`.

You can choose between the precooked model or your trained model with the python variable `precooked` in the cell below.

In [31]:
# Tabular Classification
PRECOOK_TABLE_CLASSIFICATION_MODEL = '[not-supported-yet]'
PRECOOK_MODEL = PRECOOK_TABLE_CLASSIFICATION_MODEL

# Precooked flag
precook = False

if precook:
    model_to_deploy_name = PRECOOK_MODEL
else:
    model_to_deploy = pipeline_response.model_to_upload
    model_to_deploy_name = model_to_deploy.name

print("model_to_deploy:", model_to_deploy_name)

model_to_deploy: projects/759209241365/locations/us-central1/models/3146256920927535104


## Evaluate the model

Now let's find out how good the model service believes your model is. As part of training, some portion of the dataset was set aside as the test (holdout) data, which is used by the pipeline service to evaluate the model.

### List evaluations for all slices

Use this helper function `list_model_evaluations`, which takes the parameter:

- `name`: The AI Platform (Unified) fully qualified model identifier.

This helper function uses the AI Platform (Unified) model client service, and calls the method `list_model_evaluations`, which takes the same parameter. The response object from the call is a list, where each element is an evaluation metric.

For each evaluation -- you probably only have one, we then print all the key names for each metric in the evaluation, and for a small set (`logLoss` and `auPrc`) we print the result.

In [32]:
def list_model_evaluations(name):
    response = clients['model'].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print('logloss', metrics['logLoss'])
        print('auPrc', metrics['auPrc'])

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_name)

model_evaluation
 name: projects/759209241365/locations/us-central1/models/3146256920927535104/evaluations/2882682461320025851
 metrics_schema_uri: gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml
auRoc
logLoss
confusionMatrix
confidenceMetrics
auPrc
logloss 0.017979896
auPrc 1.0


### Get evaluations for a slice

Now, let's use the AI Platform (Unified) fully qualified identifier for an evaluation to get just that specific evaluation. Use the last evaluation (`last_evaluation`) from your previous list of evaluations as an example.

Use this helper function `model_evaluation`, which takes as a parameter:

- `name`: The AI Platform (Unified) fully qualified identifier for the specific model evaluation.

The helper function uses the model client service and calls the method `get_model_evaluation`, with the parameter:

- `name`: The AI Platform (Unified) fully qualified identifier for the specific model evaluation.

Next, print the entire evaluation data -- which may seem at first somewhat verbose.

In [33]:
def model_evaluation(name):
    response = clients['model'].get_model_evaluation(name=name)
    print("response")
    print(" name:", response.name)
    print(" metrics_schema_uri:", response.metrics_schema_uri)
    print(" metrics:", json_format.MessageToDict(response._pb.metrics))
    print(" create_time:", response.create_time)
    print(" slice_dimensions:", response.slice_dimensions)
    model_explanation = response.model_explanation
    print(" model_explanation")
    mean_attributions = model_explanation.mean_attributions
    for mean_attribution in mean_attributions:
        print("  mean_attribution")
        print("   baseline_output_value:", mean_attribution.baseline_output_value)
        print("   instance_output_value:", mean_attribution.instance_output_value)
        print(
            "   feature_attributions:",
            json_format.MessageToDict(mean_attribution._pb.feature_attributions),
        )
        print("   output_index:", mean_attribution.output_index)
        print("   output_display_name:", mean_attribution.output_display_name)
        print("   approximation_error:", mean_attribution.approximation_error)


model_evaluation(last_evaluation)

response
 name: projects/759209241365/locations/us-central1/models/3146256920927535104/evaluations/2882682461320025851
 metrics_schema_uri: gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml
 metrics: {'auRoc': 1.0, 'logLoss': 0.017979896, 'confusionMatrix': {'annotationSpecs': [{'displayName': 'setosa'}, {'displayName': 'versicolor'}, {'displayName': 'virginica'}], 'rows': [[56.0, 0.0, 0.0], [0.0, 33.0, 0.0], [0.0, 0.0, 16.0]]}, 'confidenceMetrics': [{'truePositiveCount': '105', 'falsePositiveRate': 1.0, 'recall': 1.0, 'recallAt1': 1.0, 'f1Score': 0.5, 'f1ScoreAt1': 1.0, 'precisionAt1': 1.0, 'falsePositiveCount': '210', 'precision': 0.33333334}, {'precisionAt1': 1.0, 'confidenceThreshold': 0.5, 'precision': 1.0, 'truePositiveCount': '105', 'recall': 1.0, 'recallAt1': 1.0, 'f1Score': 1.0, 'trueNegativeCount': '210', 'f1ScoreAt1': 1.0}, {'f1Score': 0.96039605, 'trueNegativeCount': '210', 'f1ScoreAt1': 0.96039605, 'precisionAt1': 1.0, 'confidenceThresho

## Deploy the model

Let's now deploy the trained AI Platform (Unified) model you created with AutoML. This requires two steps:

1. Create an endpoint for deploying the model to.

2. Deploy the model to the endpoint.

### Create an endpoint

Use this helper function `create_endpoint` to create an endpoint to deploy the model to for serving predictions, with the parameter:

- `display_name`: A human readable name for the endpoint.

The helper function uses the endpoint client service and calls the method `create_endpoint`, which takes the parameter:

- `display_name`: A human readable name for the endpoint.

Creating an endpoint returns a long running operation, since it may take a few moments to provision the endpoint for serving. We call `response.result()`, which is a synchronous call and will return when the endpoint is ready. The helper function returns the AI Platform (Unified) fully qualified identifier for the endpoint -- `response.name`.

In [34]:
ENDPOINT_NAME = "iris_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients['endpoint'].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result.name


endpoint_name = create_endpoint(ENDPOINT_NAME)

Long running operation: projects/759209241365/locations/us-central1/endpoints/3397754812618506240/operations/1426000610527805440
result
 name: projects/759209241365/locations/us-central1/endpoints/3397754812618506240
 display_name: 
 description: 
 labels: {}
 create_time: None
 update_time: None


### Deploy model to the endpoint

Use this helper function `deploy_model` to deploy the model to the endpoint you created for serving predictions, with the parameters:

- `model`: The AI Platform (Unified) fully qualified model identifier of the model to upload (deploy) from the training pipeline.
- `deploy_model_display_name`: A human readable name for the deployed model.
- `endpoint`: The AI Platform (Unified) fully qualified endpoint identifier to deploy the model to.

The helper function uses the endpoint client service and calls the method `deploy_model`, which takes the parameters:

- `endpoint`: The AI Platform (Unified) fully qualified endpoint identifier to deploy the model to.
- `deployed_model`: The requirements for deploying the model.
- `traffic_split`: Percent of traffic at endpoint that goes to this model.

Let's now dive deeper into the `deployed_model` parameter. This parameter is specified as a Python dictionary with the minimum required fields:

- `model`: The AI Platform (Unified) fully qualified model identifier of the (upload) model to deploy.
- `display_name`: A human readable name for the deployed model.
- `automatic_resources`: This refers to how many redundant compute instances (replicas). For this example, we set it to one (no replication).

Let's now dive deeper into the `traffic_split` parameter. This parameter is specified as a Python dictionary. This might at first be a tad bit confusing. Let me explain, you can deploy more than one instance of your model to an endpoint, and then set how much (percent) goes to each instance. 

Why would you do that? Perhaps you already have a previous version deployed in production -- let's call that v1. You got better model evaluation on v2, but you don't know for certain that it is really better until you deploy to production. So in the case of traffic split, you might want to deploy v2 to the same endpoint as v1, but it only get's say 10% of the traffic. That way, you can monitor how well it does without disrupting the majority of users -- until you make a final decision.

The method returns a long running operation `response`. We will wait sychronously for the operation to complete by calling the `response.result()`, which will block until the model is deployed. If this is the first time a model is deployed to the endpoint, it may take a few additional minutes to complete provisioning of resources.

In [35]:
DEPLOYED_NAME = "iris_deployed-" + TIMESTAMP


def deploy_model(model, deployed_model_display_name, endpoint):

    # key '0' assigns traffic for the newly deployed model
    # Traffic percentage values must add up to 100
    # Leave dictionary empty if endpoint should not accept any traffic
    traffic_split = {"0": 100}

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        # AutoML Vision models require `automatic_resources` field
        # Other model types may require `dedicated_resources` field instead
        # "automatic_resources": {"min_replica_count": 1, "max_replica_count": 1},
        "dedicated_resources": {
            "min_replica_count": 1,
            "machine_spec": {
                "machine_type": "n1-standard-2",
                # Accelerators can be used only if the model specifies a GPU image.
                # 'accelerator_type': aiplatform.AcceleratorType.NVIDIA_TESLA_K80,
                # 'accelerator_count': 1,
            },
        },
    }

    response = clients['endpoint'].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split)

    print("Long running operation:", response.operation.name)
    result = response.result(timeout=1800)
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_name, DEPLOYED_NAME, endpoint_name)

Long running operation: projects/759209241365/locations/us-central1/endpoints/3397754812618506240/operations/7526126305801142272
result
 deployed_model
  id: 4822862619704557568
  model: 
  display_name: 
  create_time: None


## Make a prediction request

Let's now do a prediction to your deployed model. 

In [36]:
INSTANCE = {"petal_length": '1.4', "petal_width": '1.3', "sepal_length": '5.1', "sepal_width": '2.8'}

Ok, now you have a test item. Use this helper function `predict_item`, which takes the parameters:

- `data`: The test tabular entry data item.
- `endpoint`: The AI Platform (Unified) fully qualified identifier for the endpoint where the model was deployed.
- `parameters_dict`: Additional parameters for serving -- in our case we will pass None.

This function uses the prediction client service and calls the `predict` method with the parameters:

- `endpoint`: The AI Platform (Unified) fully qualified identifier for the endpoint where the model was deployed.
- `instances`: A list of instances (data items) to predict.
- `parameters`: Additional parameters for serving -- in our case we will pass None.

Since the `predict()` service can take multiple items (instances), you send your single test item as a list of one test item. As a final step, you package the instances list into Google's protobuf format -- which is what we pass to the `predict()` service.

The `response` object returns a list, where each element in the list corresponds to the corresponding image in the request. You will see in the output for each prediction -- in this case there is just one:

- Confidence level in the prediction.
- The predicted label.

In [37]:
def predict_item(data, endpoint, parameters_dict):
    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [data]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients['prediction'].predict(endpoint=endpoint, instances=instances, parameters=parameters)
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        # See gs://google-cloud-aiplatform/schema/predict/prediction/classification.yaml for the format of the predictions.
        print(" prediction:", dict(prediction))


predict_item(INSTANCE, endpoint_name, None)

response
 deployed_model_id: 4822862619704557568
predictions
 prediction: {'scores': [number_value: 0.4760102927684784
, number_value: 0.0001645536249270663
, number_value: 0.5238251090049744
], 'classes': [string_value: "versicolor"
, string_value: "virginica"
, string_value: "setosa"
]}


## Undeploy the model

Let's now undeploy our model from the serving endpoint. Use this helper function `undeploy_model`, which takes the parameters:

- `deployed_model_id`: The model deployment identifier returned by the endpoint service when the model was deployed.
- `endpoint`: The AI Platform (Unified) fully qualified identifier for the endpoint where the model is deployed.

This function uses the endpoint client service and calls the method `undeploy_model`, with the parameters:

- `deployed_model_id`: The model deployment identifier returned by the endpoint service when the model was deployed.
- `endpoint`: The AI Platform (Unified) fully qualified identifier for the endpoint where the model is deployed.
- `traffic_split`: How to split traffic among the remaining deployed models on the endpoint.

Since this is the only deployed model on the endpoint, you can leave `traffic_split` empty by setting it to {}.

In [38]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients['endpoint'].undeploy_model(endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={})
    print(response)


undeploy_model(deployed_model_id, endpoint_name)

# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Model
- Endpoint
- Cloud Storage Bucket

In [ ]:
delete_dataset = True
delete_model = True
delete_endpoint = True
delete_bucket = True

# Delete the dataset using the AI Platform (Unified) fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients['dataset'].delete_dataset(name=dataset['name'])
except Exception as e:
    print(e)
    
# Delete the model using the AI Platform (Unified) fully qualified identifier for the model
try:
    if delete_model:
        clients['model'].delete_model(name=model_to_deploy_name)
except Exception as e:
    print(e)
    
# Delete the endpoint using the AI Platform (Unified) fully qualified identifier for the endpoint
try:
    if delete_endpoint:
        clients['endpoint'].delete_endpoint(name=endpoint_name)
except Exception as e:
    print(e)

if delete_bucket and 'BUCKET_NAME' in globals():
    ! gsutil rm -r gs://$BUCKET_NAME